In [1]:
# Set up data collection foundation

CREATURE_NAME=[]
CREATURE_CAT=[]
SEASON_EP=[]
SHOW_NAME=[]

TABLE_COLUMNS = ['creature_name', 'creature_cat', 'season_ep', 'show_name']

DATABASE = {'creature_name':CREATURE_NAME, 'creature_cat':CREATURE_CAT, 'season_ep':SEASON_EP, 'show_name':SHOW_NAME}

VD_URL = 'http://vampirediaries.wikia.com/wiki/Category:Species'

In [2]:
# Libraries

from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from urllib import request

In [3]:
# Convert html to a more user friendly format

def soupTheLink(url):
    html = request.urlopen(url).read().decode('utf8')
    return soup(html, 'lxml')

In [4]:
# Scrape a list of links of the creature categories

def collectCatLinks(url):
    categoryLinks = []
    
    souped = soupTheLink(url)
    subcategories = souped.find(id='mw-subcategories')
    gallery = subcategories.find(class_='category-gallery-room1')
    for listItem in gallery.find_all(class_='category-gallery-item'):
        link = listItem.find('a', href = True)
        try:
            fullLink = link['href']
            categoryLinks.append(fullLink)
        except TypeError:
            pass
    return categoryLinks

In [5]:
# Find the number of pages of each category

def findNumCatPages(url):
    souped = soupTheLink(url)
    pages = 0
    for subcategories in souped.find_all(id='mw-subcategories'):
        for pagination in subcategories.find_all(class_='wikia-paginator'):
            for listItem in pagination.find_all('li'):
                pages += 1
    return pages

In [6]:
# Jump into the list of links to the creatures based on the categories

def intoCategoryLinks(catLinks):
    currentCategory=''
    
    for mainLink in catLinks:
        currentCategory = mainLink
        print('current category: ', currentCategory)
        numPages = findNumCreaturePages(mainLink) - 1
        if numPages == 0:
            pass
        else:
            for i in range(1,numPages):
                url = mainLink  + '?page=' + str(i)
                collectCreatureLinks(url, currentCategory)

In [7]:
# Find the number of pages in each category

def findNumCreaturePages(url):
    souped = soupTheLink(url)
    pages = 0
    subcategories = souped.find(id='mw-pages')
    pagination = subcategories.find(class_='wikia-paginator')
    try:
        for listItem in pagination.find_all('li'):
            pages += 1
    except AttributeError:
        pass
    return pages

In [8]:
# Compile a list of links to each individual creature

def collectCreatureLinks(url, currentCat):
    souped = soupTheLink(url)
    subcategories = souped.find(id='mw-pages')
    for listItem in subcategories.find_all(class_='category-gallery-item'):
        link = listItem.find('a', href = True)
        try:
            fullLink = link['href']
            findCategory(currentCat)
            intoCreatureLinks(fullLink)
        except TypeError:
            pass

In [9]:
# Import creature category to database

def findCategory(url):
    category = str(url[46:])
    DATABASE['creature_cat'].append(category)

In [11]:
# Import the rest of the data to the database

def intoCreatureLinks(creatureUrl):
    print('starting intoCreatureLinks: ', creatureUrl)
    souped = soupTheLink(creatureUrl)

    #collect creature name
    name = creatureUrl[37:]
    DATABASE['creature_name'].append(name)

    #collect show name
    DATABASE['show_name'].append('Vampire Diaries')
    
    try:
        #collect season appearances
        currentSeason = 0
        seasonLink = ''
        body = souped.find(id='mw-content-text')
        print('found body: ', body != None)
        SEASON_EP=[]

        heading = body.find(id='Appearances')
        print('found heading: ', heading != None)

        if heading == None:
            pass
        else:
            parent = heading.find_parent()
            tables = parent.find_next_sibling('table')
            print('found tables: ', tables != None)

            checkSeasons = tables.find('dl')

            if checkSeasons == None:
                for seasons in tables.find_all('b'):
                    single=[]
                    seasonDegree = 0
                    print('found a b season')
                    seasonLink = seasons.find('a', href=True)
                    season = seasons.get_text()
                    season = season[7:]
                    print('season ', season)
                    single.append(season)
                    parent = seasons.find_parent()
                    print('found parent: ', parent != None)
                    episodes = parent.find_next_sibling('ul')
                    for episode in episodes.find_all('li'):
                        print('found an episode')
                        seasonDegree += 1
                    single.append(seasonDegree)
                    SEASON_EP.append(single)
            else:
                for seasons in tables.find_all('dl'):
                    single=[]
                    seasonDegree = 0
                    print('found a dl season')
                    seasonLink = seasons.find('a', href=True)
                    season = seasons.get_text()
                    season = season[7:]
                    print('season ', season)
                    single.append(season)
                    episodes = seasons.find_next_sibling('ul')
                    for episode in episodes.find_all('li'):
                        print('found an episode')
                        seasonDegree += 1
                    single.append(seasonDegree)
                    SEASON_EP.append(single)


        if SEASON_EP == []:
            DATABASE['season_ep'].append('None')   
        else:
            DATABASE['season_ep'].extend([SEASON_EP])

    except AttributeError:
        DATABASE['season_ep'].append('None')   




In [12]:
# Run program

numPages = findNumCatPages(VD_URL) - 1
for i in range(1,numPages):
    url = VD_URL + '?page=' + str(i)
    print('mainLink: ', url)
    

    categoryLinks = collectCatLinks(url)
    intoCategoryLinks(categoryLinks)

mainLink:  http://vampirediaries.wikia.com/wiki/Category:Species?page=1
current category:  http://vampirediaries.wikia.com/wiki/Category:Supernatural
starting intoCreatureLinks:  http://vampirediaries.wikia.com/wiki/Niklaus_Mikaelson
found body:  True
found heading:  True
found tables:  True
found a b season
season  Two
found parent:  True
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found a b season
season  Three
found parent:  True
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found a b season
season  Four
found parent:  True
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an episode
found an epis

In [13]:
# Ensure that database lists are of the same length

print('1: ', len(CREATURE_NAME), '2: ', len(CREATURE_CAT), '3: ', len(SEASON_EP), '4: ', len(SHOW_NAME), )

1:  802 2:  802 3:  802 4:  802


In [15]:
# Save dataframe to file

pd.DataFrame(DATABASE).T.reset_index().to_csv('vampireDiariesCreatures.csv', header=True)